# Bioinformatics Term Project
## Artificial Neural Networks and Ranking Approach for Probe Selection and Classification of Microarray Data

### Onur Poyraz, Buse Buz

In this project we deal with the Acute Leukemia Classification problem. We have the data for the 72 patient which includes 7129 different gene type and the type of the acute leukemia. Here we apply different models to classify these patients according to their gene map.

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from random import seed
from random import random

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture, BayesianGaussianMixture
from scipy.stats import multivariate_normal

from sklearn import svm

from sklearn.naive_bayes import GaussianNB

from sklearn.gaussian_process import GaussianProcessClassifier

The below code includes Multilayer Perceptron model Neural Network code which implemented for this project.

In [2]:
# random weights
def initialize_network(inputs, hidden, outputs):
    network = list()
    hidden_layer = [{'weights':[random() for i in range(inputs + 1)]} for i in range(hidden)]
    network.append(hidden_layer)
    output_layer = [{'weights':[random() for i in range(hidden + 1)]} for i in range(outputs)]
    network.append(output_layer)
    return network

# calculate net j
def findsum(weights, inputs):
    netj = weights[-1]
    for i in range(len(weights)-1):
        netj += weights[i] * inputs[i]
    return netj

# tanh function
def transfer(netj):
    return np.tanh(netj)

# find output j, give it as an input
def forward_propagate(network, row):
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            netj = findsum(neuron['weights'], inputs)
            neuron['output'] = transfer(netj)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs


# derivative of tanh function
def transfer_derivative(output):
    return 1.0- (output * output)

def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        # ara basamak
        if i != len(network)-1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
        # final basamak
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

def update_weights(network, row, l_rate):
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] += l_rate * neuron['delta']

def train_network(network, train, l_rate, n_epoch, n_outputs):
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            outputs = forward_propagate(network, row)
            expected = [0 for i in range(n_outputs)]
            expected[row[-1]] = 1
            sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
            backward_propagate_error(network, expected)
            update_weights(network, row, l_rate)
        #print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))

def predict(network, row):
    outputs = forward_propagate(network, row)
    return outputs.index(max(outputs))

In [3]:
df_train = pd.read_csv('./gene-expression/data_set_ALL_AML_train.csv')
df1 = [col for col in df_train.columns if "call" not in col]
df_train = df_train[df1]

df_train = df_train.T
df_train = df_train.drop(['Gene Description','Gene Accession Number'],axis=0)
df_train.index = pd.to_numeric(df_train.index)
df_train.sort_index(inplace=True)

df_train['cat'] = list(pd.read_csv('./gene-expression/actual.csv')[:38]['cancer'])
dic = {'ALL':0,'AML':1}
df_train.replace(dic,inplace=True)
df_train

,0,1,2,3,4,5,6,7,8,9,...,7120,7121,7122,7123,7124,7125,7126,7127,7128,cat
1,-214,-153,-58,88,-295,-558,199,-176,252,206,...,511,-125,389,-37,793,329,36,191,-37,0
2,-139,-73,-1,283,-264,-400,-330,-168,101,74,...,837,-36,442,-17,782,295,11,76,-14,0
3,-76,-49,-307,309,-376,-650,33,-367,206,-215,...,1199,33,168,52,1138,777,41,228,-41,0
4,-135,-114,265,12,-419,-585,158,-253,49,31,...,835,218,174,-110,627,170,-50,126,-91,0
5,-106,-125,-76,168,-230,-284,4,-122,70,252,...,649,57,504,-26,250,314,14,56,-25,0
6,-138,-85,215,71,-272,-558,67,-186,87,193,...,1221,-76,172,-74,645,341,26,193,-53,0
7,-72,-144,238,55,-399,-551,131,-179,126,-20,...,819,-178,151,-18,1140,482,10,369,-42,0
8,-413,-260,7,-2,-541,-790,-275,-463,70,-169,...,629,-86,302,23,1799,446,59,781,20,0
9,5,-127,106,268,-210,-535,0,-174,24,506,...,980,6,177,-12,758,385,115,244,-39,0
10,-88,-105,42,219,-178,-246,328,-148,177,183,...,986,26,101,21,570,359,9,171,7,0


The below data extracts the test data from the excels.

In [4]:
df_test = pd.read_csv('./gene-expression/data_set_ALL_AML_independent.csv')
df1 = [col for col in df_test.columns if "call" not in col]
df_test = df_test[df1]

df_test = df_test.T
df_test = df_test.drop(['Gene Description','Gene Accession Number'],axis=0)
df_test.index = pd.to_numeric(df_test.index)
df_test.sort_index(inplace=True)

df_test['cat'] = list(pd.read_csv('./gene-expression/actual.csv')[38:]['cancer'])
dic = {'ALL':0,'AML':1}
df_test.replace(dic,inplace=True)

In this part we apply the Principal Component Analysis (PCA) to the data since it has 7129 different features for this classification. This is the one of the most important part of this project since the reducing the dimension of the data is very important for the analysis of the genes.

In [5]:
X_std_train = StandardScaler().fit_transform(df_train.drop('cat',axis=1))
X_std_test = StandardScaler().fit_transform(df_test.drop('cat',axis=1))

pca_element = 30
applied_pca = PCA(n_components = pca_element)

Y_sklearn_train = applied_pca.fit_transform(X_std_train)
Y_sklearn_test = applied_pca.fit_transform(X_std_test)

In [6]:
X_reduced_train = Y_sklearn_train
X_reduced_test = Y_sklearn_test
train = pd.DataFrame(X_reduced_train)
test = pd.DataFrame(X_reduced_test)

train['cat'] =  df_train['cat'].reset_index().cat
test['cat'] =  df_test['cat'].reset_index().cat

training = train.as_matrix(columns = None)
training_list = training.tolist()

tester = test.as_matrix(columns = None)
tester_list = tester.tolist()

In [7]:
for i in range(len(training_list)):
    training_list[i][pca_element] = int(training_list[i][pca_element])
for i in range(len(tester_list)):
    tester_list[i][pca_element] = int(tester_list[i][pca_element])

In [8]:
np.savetxt('test.txt', tester, delimiter=',', fmt = '%.5f')

### Implementation of the Neural Network

In [9]:
seed(1)
prediction = np.zeros(34)
dataset = training_list
dataset_test = tester_list
n_inputs = len(dataset[0]) - 1
n_outputs = len(set([row[-1] for row in dataset]))
network = initialize_network(n_inputs, 2, n_outputs)
train_network(network, dataset, 0.01, 2000, n_outputs)
count = 0
for counter, row in enumerate(dataset_test):
    pred= predict(network, row)
    prediction[counter] = pred
    #print('Expected=%d, Got=%d' % (row[-1], prediction))
    if (row[-1]==pred):
        count +=1
        
print (count)

22


### Implementation of SVM Algorithm

In [10]:
svm_clf = svm.SVC()
svm_clf.fit(training[:,0:30], training[:,30])

svm_predict = svm_clf.predict(tester[:,0:30])

In [11]:
svm_predict


array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [12]:
count = 0
for i in range(34):
    if test['cat'][i] == svm_predict[i]:
        count = count + 1
        
print (count)

20


### Implementation of Gaussian Process Classifier

In [13]:
GP_clf = GaussianProcessClassifier()
GP_clf.fit(training[:,0:30], training[:,30])

GP_predict = GP_clf.predict(tester[:,0:30])

In [14]:
GP_predict

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [15]:
count = 0
for i in range(34):
    if test['cat'][i] == GP_predict[i]:
        count = count + 1
        
print (count)

20


### Implementation of KMeans Algorithm

In [16]:
number_of_clusters = 2
kmeans = KMeans(n_clusters = number_of_clusters, random_state = 0).fit(training[:,0:30])
node_kmeans = kmeans.labels_
center_kmeans = kmeans.cluster_centers_
kmeans_predict = kmeans.predict(tester[:,0:30])

In [17]:
kmeans_predict

array([1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1], dtype=int32)

In [18]:
count = 0
for i in range(34):
    if train['cat'][i] == kmeans_predict[i]:
        count = count + 1
        
print (count)

15


### Implementation of Gaussian Naive Bayes Algorithm

In [19]:
NB_clf = GaussianNB()
NB_clf.fit(training[:,0:30], training[:,30])

NB_predict = NB_clf.predict(tester[:,0:30])

In [20]:
NB_predict

array([ 1.,  1.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,
        1.,  0.,  0.,  0.,  1.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [21]:
count = 0
for i in range(34):
    if test['cat'][i] == NB_predict[i]:
        count = count + 1
        
print (count)

15


# Evaluation

In [22]:
length = 34
confmat = np.zeros((2,4))
for i in range(length):
    if(tester[i,30] == prediction[i]):
        if (int(prediction[i])==0):
            confmat[int(tester[i,30]),0] = confmat[int(tester[i,30]),0] + 1
        if (int(prediction[i])==1):
            confmat[int(prediction[i]),3] = confmat[int(prediction[i]),3] + 1
        #confmat[-int(tester[i,30]),3] = confmat[-int(tester[i,30]),3] +1
        #confmat[-int(prediction[i]),3] = confmat[-int(prediction[i]),3] +1  
    else:
        if (int(prediction[i])==0):
            confmat[int(tester[i,30]),1] = confmat[int(tester[i,30]),1] +1
        if (int(prediction[i])==1):
            confmat[int(prediction[i]),2] = confmat[int(prediction[i]),2] +1
        #confmat[-c(tester[i,30],prediction[i]), 4] =confmat[-c(tester[i,30],prediction[i]), 4] +1

In [25]:
confmat

array([[ 20.,   0.,   0.,   0.],
       [  0.,  12.,   0.,   2.]])